### https://docs.juliacn.com/latest/manual/metaprogramming/#man-quote-node

In [3]:
Meta.quot(Expr(:$, :(1+2)))

:($(Expr(:quote, :($(Expr(:$, :(1 + 2)))))))

In [ ]:
:(
    $Expr(
            :quote
            :(
                $(Expr(:$, :(1+2))) # 求值表达式
             ) # 原本表达式
         ) # Expr 中的参数为 符号、表达式、字面量. :(1+2)不是字面量3,就要外加:(), 否则会被计算
)

In [12]:
Meta.quot(Expr(:$, :(1+2))) == :($(1+2))

false

In [14]:
:($(1+2)) |> display

eval(:($(1+2))) |> display

eval(3) |> display

3

3

3

In [7]:
Meta.quot(Expr(:$, :(1+2))) == :($(:(1+2)))

false

In [8]:
:($(:(1+2)))

:(1 + 2)

In [9]:
eval(Meta.quot(Expr(:$, :(1+2)))) ==

eval(
    :(
        $(:(1+2))
     )
    )

true

表达式支持插值符号 `$`。但是，在某些情况下，需要在不执行插值的情况下引用代码。 这种引用还没有语法，但在内部表示为 `QuoteNode` 类型的对象：

In [16]:
eval(Meta.quot(Expr(:$, :(1+2)))) |> display

eval(QuoteNode(Expr(:$, :(1+2)))) |> display

3

:($(Expr(:$, :(1 + 2))))

解析器为简单的引用项（如符号）生成 `QuoteNode`：

In [ ]:
dump(Meta.parse(":x"))

QuoteNode
  value: Symbol x


In [ ]:
using TreeView

In [ ]:
TreeView.walk_tree(Meta.parse(":x"))

MethodError: MethodError: no method matching walk_tree(::QuoteNode)
The function `walk_tree` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  walk_tree(!Matched::Expr)
   @ TreeView C:\Users\TCY\.julia\packages\TreeView\cUQUy\src\tree.jl:50
  walk_tree(!Matched::Expr, !Matched::Any)
   @ TreeView C:\Users\TCY\.julia\packages\TreeView\cUQUy\src\tree.jl:50


In [ ]:
QuoteNode(:x)

:(:x)

In [ ]:
Meta.parse(":x") == :(:x) == QuoteNode(:x)

true

In [ ]:
@show Meta.quot(3) == :(3)
@show eval(Meta.quot(3)) == eval(:(3))

Meta.quot(3) == $(QuoteNode(3)) = false
eval(Meta.quot(3)) == eval($(QuoteNode(3))) = true


true

### https://stackoverflow.com/a/26414175

In [ ]:
# https://stackoverflow.com/a/26414175
# Better to use Meta.quot for the instead of 
# QuoteNode, which is a Julia internal. 

str = "Obj1"
Meta.quot(Symbol("create$str")) |> display
Expr(:quote, Symbol("create$str")) |> display
QuoteNode(Symbol("create$str")) |> display

dump(:(a + b + :c))

:(:createObj1)

:(:createObj1)

:(:createObj1)

Expr
  head: Symbol call
  args: Array{Any}((4,))
    1: Symbol +
    2: Symbol a
    3: Symbol b
    4: QuoteNode
      value: Symbol c


### JeffBezanson


**JeffBezanson:**<br>
https://github.com/JuliaLang/julia/issues/6104#issuecomment-37260078<br><br>

`Meta.quot(x)` just calls `Expr(:quote, x)`. It's a shorthand. <br>
`Meta.quot(x)`是`Expr(:quote, x)`的速记形式<br><br>

`QuoteNode` is only used in the internal AST representation, not by macros. It's safe to ignore unless you work on later stages of the compiler.<br>
`QuoteNode` 仅在内部AST表示中使用，而不用于宏。忽略它是安全的，除非您在编译器的后期阶段工作。

@vtjnash: But what is the difference, since now `QuoteNode` can be constructed directly? It seems like `Expr(:quote)` is redundant and thus unnecessary.<br>
但是现在`QuoteNode`可以直接构造了，这有什么区别呢？`Expr（:quote）`似乎是多余的，因此是不必要的。<br><br>
**JeffBezanson:**<br>
https://github.com/JuliaLang/julia/issues/6104#issuecomment-37262662<br><br>

There is actually a difference in behavior come to think of it. `QuoteNode` is **very primitive and only includes a literal value** in an AST. When `QuoteNode(x)` is evaluated, the result is just `x`, the value stored inside the `QuoteNode`. **A quote expression does `$` substitution.**<br>
想想看，这实际上是行为上的不同。`QuoteNode(x)`是非常原始的，并且在AST中只包含一个字面量。当求值时，结果只是`x` (存储在了`QuoteNode`)。表达式做代换。<br><br>

The idea was that for heavy metaprogramming you'd do using Base.Meta.<br>
这个想法是，对于重度的元编程，你会使用using Base.Meta。

https://github.com/JuliaLang/julia/issues/16560#issue-156537707

In [ ]:
# https://github.com/JuliaLang/julia/issues/16560#issue-156537707
dump(:(x=:a))

Expr
  head: Symbol =
  args: Array{Any}((2,))
    1: Symbol x
    2: QuoteNode
      value: Symbol a


In [ ]:
macro uv(expr)
    _uv(expr)
end

function _uv(expr)
    show(expr)
    println("\n")

    show(eval(expr))
    println("\n")

    dump(expr)
    expr
end

_uv (generic function with 1 method)

In [ ]:
@macroexpand @uv(:(x=:a))

:($(Expr(:quote, :(x = :a))))

:(x = :a)

Expr
  head: Symbol quote
  args: Array{Any}((1,))
    1: Expr
      head: Symbol =
      args: Array{Any}((2,))
        1: Symbol x
        2: QuoteNode
          value: Symbol a


:($(Expr(:copyast, :($(QuoteNode(:(x = :a)))))))

### 如下笔者测试，不知是否有意义

In [ ]:
dump(:(3))

Int64 3


In [ ]:

dump($(QuoteNode(3)))

ErrorException: syntax: "$" expression outside quote around e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X25sZmlsZQ==.jl:2

In [ ]:
dump(quote $(QuoteNode(3)) end)

Expr
  head: Symbol block
  args: Array{Any}((2,))
    1: LineNumberNode
      line: Int64 1
      file: Symbol e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X14sZmlsZQ==.jl
    2: QuoteNode
      value: Int64 3


In [ ]:
dump(:($(QuoteNode(3))))
# :($(QuoteNode(3)) 与 QuoteNode(3) 等价

QuoteNode
  value: Int64 3


In [ ]:
dump(QuoteNode(3))

QuoteNode
  value: Int64 3


In [2]:
a=1
dump(:($(QuoteNode(a))))

QuoteNode
  value: Int64 1


In [3]:
dump(:($(QuoteNode(:a))))

QuoteNode
  value: Symbol a


In [5]:
@show :(:x) == QuoteNode(:x)
x = 0
@show :x ==QuoteNode(x)
@show eval(:x) == eval(QuoteNode(x))

$(Expr(:quote, :(:x))) == QuoteNode(:x) = true
:x == QuoteNode(x) = false
eval(:x) == eval(QuoteNode(x)) = true


true

In [6]:
@show :0 ==QuoteNode(x)
@show eval(:0) == eval(QuoteNode(x))
$(QuoteNode(0))

$(QuoteNode(0)) == QuoteNode(x) = false
eval($(QuoteNode(0))) == eval(QuoteNode(x)) = true


ErrorException: syntax: "$" expression outside quote around e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X34sZmlsZQ==.jl:3

In [7]:
@show fieldnames(QuoteNode)

fieldnames(QuoteNode) = (:value,)


(:value,)

In [8]:
dump(:a)

Symbol a


In [9]:
eval(QuoteNode(:a))

:a

### https://discourse.julialang.org/t/create-quotenode-in-expr/1319/5

In [1]:
function f1a()
    e1 = Expr(:tuple)
    e1.args = [Core.QuoteNode(Symbol("u3")), "./u1.so"]

    e2 = :Float64
    
    e3 = Expr(:tuple)
    e3.args = [:Float64]
    
    e4 = :x
    
    e5 = Expr(:ccall)
    e5.args = [e1,e2,e3,e4]
    e5
end
f1a_ex = f1a()

:($(Expr(:ccall, :((:u3, "./u1.so")), :Float64, :((Float64,)), :x)))

In [5]:
dump(f1a_ex)

Expr
  head: Symbol ccall
  args: Array{Any}((4,))
    1: Expr
      head: Symbol tuple
      args: Array{Any}((2,))
        1: QuoteNode
          value: Symbol u3
        2: String "./u1.so"
    2: Symbol Float64
    3: Expr
      head: Symbol tuple
      args: Array{Any}((1,))
        1: Symbol Float64
    4: Symbol x


In [ ]:
Expr(:ccall, 
     Expr(:tuple, QuoteNode(:u3), "./u1.so"), 
     :Float64,
     Expr(:tuple,:Float64)
     :x
)

In [ ]:
f1a_ex_format = 
:(
    $(
        Expr(:ccall, 
             :((:u3, "./u1.so")), 
             :Float64,
             :((Float64,)),
             :x)
     )
) 

In [3]:
:(
    ccall((:u3,"./u1.so"),Float64,(Float64,),x)
 )

false

Tamas_Papp reply:<br>
I believe that you should use Meta.quot(Symbol("u3")) instead. See [the discussion at](https://github.com/JuliaLang/julia/issues/6104)
